<a href="https://colab.research.google.com/github/zacSimo/PysparkAdvanced/blob/main/pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
! pip install pyspark

In [152]:
import pyspark.sql

from pyspark.sql import SparkSession

In [153]:
spark = SparkSession.builder.appName('Practice').getOrCreate()


In [154]:
spark.sparkContext.setLogLevel("ERROR")

In [155]:
sd_df = spark.read.csv('/kaggle/input/fifa-18-demo-player-dataset/PlayerAttributeData.csv', header=True, inferSchema=True)


In [156]:
sd_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Aggression: string (nullable = true)
 |-- Agility: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- Ball control: string (nullable = true)
 |-- Composure: string (nullable = true)
 |-- Crossing: string (nullable = true)
 |-- Curve: string (nullable = true)
 |-- Dribbling: string (nullable = true)
 |-- Finishing: string (nullable = true)
 |-- Free kick accuracy: string (nullable = true)
 |-- GK diving: string (nullable = true)
 |-- GK handling: string (nullable = true)
 |-- GK kicking: string (nullable = true)
 |-- GK positioning: string (nullable = true)
 |-- GK reflexes: string (nullable = true)
 |-- Heading accuracy: string (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Interceptions: string (nullable = true)
 |-- Jumping: string (nullable = true)
 |-- Long passing: string (nullable = true)
 |-- Long shots: string (nullable = true)
 |-- Marking: string (nullable = 

In [157]:
sd_df.count()

17981

#####Apache spark memory

In [158]:
from pyspark.sql.functions import *

In [159]:
sd_broad_df = broadcast(sd_df)

In [160]:
# UDF

def my_func(code: str) -> str:

  return bdData.value.get(code)




In [161]:
import pyspark.sql.functions as f

spark.udf.register("my_function", my_func, StringType())

<function __main__.my_func(code: str) -> str>

In [162]:
# see how many cores available for spark engine

spark.sparkContext.defaultParallelism

4

In [163]:
# generate ramdom data

import random

randomlist = random.sample(range(1, 40),10)

In [164]:
print(randomlist)

[23, 37, 2, 15, 14, 10, 13, 12, 16, 29]


In [165]:
# parallelize and take it into 4 partitions

rdd1 = spark.sparkContext.parallelize(randomlist, 4)

In [166]:
rdd1

ParallelCollectionRDD[254] at readRDDFromFile at PythonRDD.scala:289

In [167]:
# show exactly the same data as created

rdd1.collect()

[23, 37, 2, 15, 14, 10, 13, 12, 16, 29]

In [168]:
# to see the numbers of partitions

rdd1.getNumPartitions()

4

In [169]:
# to see the data in partitions

rdd1.glom().collect()

[[23, 37], [2, 15], [14, 10], [13, 12, 16, 29]]

In [170]:
rdd1.take(rdd1.count()-1)

[23, 37, 2, 15, 14, 10, 13, 12, 16]

In [171]:
rdd1.glom().collect()[3]

[13, 12, 16, 29]

In [172]:
rdd1.top(3)

[37, 29, 23]

In [173]:
rdd_map = rdd1.map(lambda x: x*2)

rdd_map.collect()

[46, 74, 4, 30, 28, 20, 26, 24, 32, 58]

In [174]:
def myfunc(x):

  if x%2 == 0:

    return x * 2

  else:

    return x*3



rdd_map_2 = rdd1.map(myfunc)

rdd_map_2.collect()

[69, 111, 4, 45, 28, 20, 39, 24, 32, 87]

In [175]:
rdd_filter = rdd1.filter(lambda x: x%2 == 0)

rdd_filter.collect()

[2, 14, 10, 12, 16]

In [176]:
# flatmap collect values into a single list

# used when we want aggregation of the result (reduce)

rdd_flatmap = rdd1.flatMap(lambda x: (x, x*10))

rdd_flatmap.collect()

[23,
 230,
 37,
 370,
 2,
 20,
 15,
 150,
 14,
 140,
 10,
 100,
 13,
 130,
 12,
 120,
 16,
 160,
 29,
 290]

In [177]:
rdd_map_3 = rdd1.map(lambda x: [x, x*10])

rdd_map_3.collect()

[[23, 230],
 [37, 370],
 [2, 20],
 [15, 150],
 [14, 140],
 [10, 100],
 [13, 130],
 [12, 120],
 [16, 160],
 [29, 290]]

In [178]:
rdd_map_3.glom().collect()

[[[23, 230], [37, 370]],
 [[2, 20], [15, 150]],
 [[14, 140], [10, 100]],
 [[13, 130], [12, 120], [16, 160], [29, 290]]]

In [179]:
print(rdd1.max())

print(rdd1.min())

print(rdd1.mean())

print(rdd1.sum())

print(rdd1.stdev())

37
2
17.1
171
9.533624704172071


In [180]:
# mapPartitions() map a function into each partitions

def my_partition_func(iterator):

  sum = 0

  for i in iterator:

    sum += i



  yield sum



rdd_map_partition = rdd1.mapPartitions(my_partition_func)

rdd_map_partition.collect()

[60, 17, 24, 70]

In [181]:
# union

rdd2 = spark.sparkContext.parallelize(random.sample(range(1,20), 10),2)

rdd2.collect()


[9, 16, 18, 8, 4, 1, 7, 13, 3, 12]

In [182]:
rdd1.union(rdd2).collect()

[23, 37, 2, 15, 14, 10, 13, 12, 16, 29, 9, 16, 18, 8, 4, 1, 7, 13, 3, 12]

In [183]:
rdd_union = rdd1.union(rdd2)

rdd_union.glom().collect()

[[23, 37],
 [2, 15],
 [14, 10],
 [13, 12, 16, 29],
 [9, 16, 18, 8, 4],
 [1, 7, 13, 3, 12]]

In [184]:
rdd1.union(rdd2).getNumPartitions()

6

In [185]:
rdd1.union(rdd2).glom().collect()

[[23, 37],
 [2, 15],
 [14, 10],
 [13, 12, 16, 29],
 [9, 16, 18, 8, 4],
 [1, 7, 13, 3, 12]]

In [186]:
rdd_inter = rdd1.intersection(rdd2)

rdd_inter.collect()

[12, 13, 16]

In [187]:
rdd_inter.glom().collect()

[[12], [13], [], [], [16], []]

In [188]:
rdd_inter.coalesce(1).glom().collect()

[[12, 13, 16]]

In [189]:
# takeSample withreplacement, num, [seed]

rdd1.takeSample(False, 5)

[23, 12, 13, 10, 16]

In [190]:
# takeOrdered

rdd1.takeOrdered(5)

[2, 10, 12, 13, 14]

In [191]:
rdd1.takeOrdered(5, key=lambda x: -x)

[37, 29, 23, 16, 15]

In [192]:
rdd1.reduce(lambda x, y: x+y)

171

In [193]:
rdd_rbk = rdd_union.map(lambda x: (x, x*x)).reduceByKey(lambda x,y: x+y)

In [194]:
rdd_rbk.toDF(schema=["key","value"]).show(vertical=False,truncate=False)

+---+-----+
|key|value|
+---+-----+
|12 |288  |
|18 |324  |
|37 |1369 |
|13 |338  |
|1  |1    |
|7  |49   |
|2  |4    |
|14 |196  |
|8  |64   |
|15 |225  |
|9  |81   |
|3  |9    |
|10 |100  |
|16 |512  |
|4  |16   |
|23 |529  |
|29 |841  |
+---+-----+



In [195]:
# sortByKey()

rdd_rbk.sortByKey().collect()

[(1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (7, 49),
 (8, 64),
 (9, 81),
 (10, 100),
 (12, 288),
 (13, 338),
 (14, 196),
 (15, 225),
 (16, 512),
 (18, 324),
 (23, 529),
 (29, 841),
 (37, 1369)]

In [196]:
rdd_rbk.countByKey()

defaultdict(int,
            {12: 1,
             18: 1,
             37: 1,
             13: 1,
             1: 1,
             7: 1,
             2: 1,
             14: 1,
             8: 1,
             15: 1,
             9: 1,
             3: 1,
             10: 1,
             16: 1,
             4: 1,
             23: 1,
             29: 1})

In [197]:
rdd_gbk = rdd_rbk.groupByKey().collect()

In [198]:
for item in rdd_gbk:

  print(item[0], list(item[1]))

12 [288]
18 [324]
37 [1369]
13 [338]
1 [1]
7 [49]
2 [4]
14 [196]
8 [64]
15 [225]
9 [81]
3 [9]
10 [100]
16 [512]
4 [16]
23 [529]
29 [841]


In [199]:
rdd_gby_1 = rdd_rbk.groupByKey()

In [200]:
import os

# os.environ["PYTHONHASHSEED"] = "false"

In [201]:


rdd_rbk.sortByKey().lookup(10)

[100]

In [202]:
rdd_rbk.cache()

PythonRDD[331] at RDD at PythonRDD.scala:53

In [203]:
  rdd_rbk.is_cached

True

In [204]:
rdd_rbk.unpersist()

PythonRDD[331] at RDD at PythonRDD.scala:53

In [205]:
# DF

spark

In [206]:
#df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/kaggle/input/fifa-18-demo-player-dataset/CompleteDataset.csv")

In [207]:
#df.printSchema()

In [208]:
#df.show(truncate=False)

In [209]:
#df.count()

In [210]:
#df.rdd.glom().collect()

In [211]:
#df.rdd.getNumPartitions()

In [212]:
#df2 = df.repartition(4)

In [213]:
#df2.rdd.getNumPartitions()

In [214]:
#df2.rdd.glom().collect()

In [215]:
#df.withColumnRenamed("_c0", "id").show()

In [216]:
#df2 = df.withColumnRenamed("_c0", "id")


In [217]:
#df2.printSchema()

In [218]:
#df2.rdd.getNumPartitions()

In [219]:
#df2 = df2.repartition(6)

In [220]:
#df2.rdd.getNumPartitions()

In [221]:
#df3 = df2.repartition(3, "Nationality")

In [222]:
#df3.rdd.glom().collect()

In [223]:
#df3.rdd.getNumPartitions()

In [224]:
#df3.show()

In [225]:
#df3.na.fill({"CAM": 10, "CB": 1}).show()

In [226]:
#df3.filter(df3["Nationality"] == "Germany").show()

In [227]:
#df3.groupBy("Nationality").count().sort("Nationality").show(50)

In [228]:
#df3.groupBy("Nationality", "Age").count().sort("Nationality","Age").show(50)

In [229]:
from pyspark.sql import *

In [230]:
# visualize with pandas

#df3_res = df3.where(df3["Overall"]>70).filter(df3["Nationality"] == "Albania").groupBy("Nationality", "Age").count().sort("Nationality","Age").toPandas()

#df3_res.plot(kind="bar", x="Nationality", y="count")


In [231]:
#df3_res_2 = df3.where(df3["Overall"]>70).groupBy("Age").count().sort("Age").toPandas()

#df3_res_2.plot(kind="bar", x="Age", y="count")


In [232]:
# UDF

def uppercase_lowercase(str):

  if str:  
      if len(str) > 10:
    
        return str.upper()
    
      else:
    
        return str.lower()
  else:
      return ""



spark.udf.register("uppercase_lowercase", uppercase_lowercase, StringType())

<function __main__.uppercase_lowercase(str)>

In [233]:
#df = spark.read.format("json").load('/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json')

In [234]:
# register df

#df.createOrReplaceTempView("fifa_table")

In [235]:
# df2.select("Name", uppercase_lowercase("Name")).show()

#spark.sql("select comments, uppercase_lowercase(comments) as upper_comments from fifa_table").show()

In [236]:
#broadcast(df)

In [237]:
#df.select("abstract", uppercase_lowercase("abstract")).show()

In [238]:
#df.printSchema()

In [239]:
#df.count()

In [240]:
#df.rdd.getNumPartitions()

In [241]:
#df.rdd.glom().collect()

In [242]:
#df.show()

In [243]:
#df.cache()

In [244]:
spark.sparkContext.defaultParallelism

4

In [245]:
#df.count()

In [246]:
import json

In [247]:
rdd_json = spark.sparkContext.textFile('/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json')

In [248]:
rdd = rdd_json.map(lambda x: json.loads(x))

In [249]:
rdd

PythonRDD[334] at RDD at PythonRDD.scala:53

In [250]:
spark.sparkContext.defaultParallelism

4

In [251]:
# rdd.persist()

In [252]:
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

In [253]:
rdd.count()

2011231

In [254]:
rdd.flatMap(lambda x: x.keys()).distinct().collect()

['comments',
 'versions',
 'abstract',
 'report-no',
 'license',
 'update_date',
 'authors',
 'journal-ref',
 'doi',
 'categories',
 'submitter',
 'authors_parsed',
 'id',
 'title']

In [255]:
rdd.map(lambda x: x['license']).distinct().collect()

[None,
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/licenses/by/3.0/']

In [256]:
# min title

min_title = rdd.map(lambda x: x['title']).distinct().reduce(lambda x,y: x if (len(x)<len(y)) else y)

print(min_title)

Q


In [257]:
# max title

max_title = rdd.map(lambda x: x['title']).distinct().reduce(lambda x,y: x if (len(x)>len(y)) else y)

print(max_title)

Investigation of the 2-body system with a rotating central body (e. g.
  earth-moon system) within the Projective Unified Field theory: the transfer
  of rotational angular momentum and energy from the central body to the
  orbital 2-body system, the tidal and the non-tidal influences (mechanical,
  general-relativistic Lense-Thirring effect and cosmological
  PUFT-contributions)


In [258]:
import re

In [259]:
# find abbreeviations with 5 or more letters in the "abstract" column

# exept : [^_! /\\:;&#,~<>]

regx = r"\((\w[^_! /\\:;&#,~?${}<>]{5,})\)"

In [260]:
print(re.search(regx, "of the Large Hadron\nCollider (LHCOPS)"))

<re.Match object; span=(29, 37), match='(LHCOPS)'>


In [261]:


rdd.filter(lambda x: re.search(regx, x['abstract'])).count()

188806

In [262]:
# rdd.filter(lambda x: re.search(regx, x['abstract'])).collect()

In [263]:
# Number of archive per month

rdd.map(lambda x: (x['update_date'][5:7], 1)).reduceByKey(lambda x,y: x+y).collect()

[('01', 134247),
 ('06', 191746),
 ('05', 296587),
 ('10', 197755),
 ('11', 297963),
 ('02', 116948),
 ('04', 117126),
 ('03', 126458),
 ('07', 122649),
 ('09', 138978),
 ('08', 138469),
 ('12', 132305)]

In [264]:
from datetime import datetime



def extract_date(line):

  d = datetime.strptime(line, '%Y-%m-%d')

  return d.month



extract_date("2008-12-20")

12

In [265]:
# Number of archive per month

rdd.map(lambda x: (extract_date(x['update_date']), 1)).reduceByKey(lambda x,y: x+y).collect()

[(1, 134247),
 (2, 116948),
 (3, 126458),
 (4, 117126),
 (5, 296587),
 (6, 191746),
 (7, 122649),
 (8, 138469),
 (9, 138978),
 (10, 197755),
 (11, 297963),
 (12, 132305)]

In [266]:
# the average number of pages

# in comments column

# extract only the number be carefull with "None" value

pages = "pages"



def extract_num_pages(comments):

  regex = r"(\d+ pages)"

  r = re.findall(regex, str(comments))

  if r:

   return int(r[0].split(' ')[0])

  return 0



extract_num_pages("Extensively revised version. In the revised version, we have\n  discussed the effect of viscosity on (quark) transverse momentum distribution\n  and on elliptic flow. It was shown that elliptic flow saturates due to\n  non-equilirium correction to the equilibrium distribution function and can\n  not be mimicked by ideal dynamics. 15 pages, 16 figures")

15

In [268]:
rdd_com = rdd.map(lambda x: (x["comments"],extract_num_pages(x["comments"]))).collect()


In [269]:
rdd.map(lambda x: extract_num_pages(x['comments'])).mean()

10.51073496778838

In [270]:
rdd_page_row = rdd.map(lambda x: (x["comments"],extract_num_pages(x["comments"]))).count()

In [271]:
rdd_page_row

2011231

In [272]:
rdd_nn_pages = rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).count()

In [273]:
rdd_nn_pages

2011231

In [274]:
rdd_filter_zero = rdd.filter(lambda x: x != 0).count()

In [275]:
rdd_filter_zero

2011231

In [276]:
# average with mean function
rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).mean()

10.51073496778838

In [277]:
# average with reduce
rdd_reduce = rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).reduce(lambda x,y: x+y)

In [278]:
print(rdd_reduce/rdd_filter_zero)

10.510734967788384


In [279]:
# using DF
df = spark.read.json("/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json")

In [280]:
df

DataFrame[abstract: string, authors: string, authors_parsed: array<array<string>>, categories: string, comments: string, doi: string, id: string, journal-ref: string, license: string, report-no: string, submitter: string, title: string, update_date: string, versions: array<struct<created:string,version:string>>]

In [281]:
df.rdd.getNumPartitions()

25

In [282]:
df.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [283]:
df.count()

2011231

In [284]:
from pyspark.sql.types import *

schema = StructType([
    StructField("authors",StringType(), True),
    StructField("categories",StringType(), True),
    StructField("license",StringType(), True),
    StructField("comments",StringType(), True),
    StructField("abstract",StringType(), True),
    StructField("versions", ArrayType(StringType()), True)
])
schema

StructType([StructField('authors', StringType(), True), StructField('categories', StringType(), True), StructField('license', StringType(), True), StructField('comments', StringType(), True), StructField('abstract', StringType(), True), StructField('versions', ArrayType(StringType(), True), True)])

In [285]:
# Binding data to a schama
df = spark.read.json("/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json", schema=schema)

In [286]:
df.show(vertical=True)

-RECORD 0--------------------------
 authors    | C. Bal\'azs, E. L... 
 categories | hep-ph               
 license    | NULL                 
 comments   | 37 pages, 15 figu... 
 abstract   |   A fully differe... 
 versions   | [{"version":"v1",... 
-RECORD 1--------------------------
 authors    | Ileana Streinu an... 
 categories | math.CO cs.CG        
 license    | http://arxiv.org/... 
 comments   | To appear in Grap... 
 abstract   |   We describe a n... 
 versions   | [{"version":"v1",... 
-RECORD 2--------------------------
 authors    | Hongjun Pan          
 categories | physics.gen-ph       
 license    | NULL                 
 comments   | 23 pages, 3 figures  
 abstract   |   The evolution o... 
 versions   | [{"version":"v1",... 
-RECORD 3--------------------------
 authors    | David Callan         
 categories | math.CO              
 license    | NULL                 
 comments   | 11 pages             
 abstract   |   We show that a ... 
 versions   | [{"version":"v

In [287]:
# df count
df.count()

2011231

In [288]:
# Missing values for "comments" and "licence" attributes
# drop
df = df.dropna(subset = ["comments"])

df.count()

1567634

In [289]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|          categories|             license|            comments|            abstract|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|              hep-ph|                NULL|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|      physics.gen-ph|                NULL| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|             math.CO|                NULL|            11 pages|  We show that a ...|[{"version":"v1",...|
|Y. H. Pong and C....|   cond-mat.mes-hall|                NULL|6 pages, 4 figure...|  We study the tw..

In [290]:
# Missing values for "comments" and "licence" attributes

# replace
df = df.fillna(value = "Unknown", subset = ["license"])
df.count()

1567634

In [291]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|          categories|             license|            comments|            abstract|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|              hep-ph|             Unknown|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|      physics.gen-ph|             Unknown| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|             math.CO|             Unknown|            11 pages|  We show that a ...|[{"version":"v1",...|
|Y. H. Pong and C....|   cond-mat.mes-hall|             Unknown|6 pages, 4 figure...|  We study the tw..

In [292]:
# renamed df to reuse in sql query
df.createOrReplaceTempView("Archive")

query = """select authors, categories from Archive
            where categories like 'math%'
"""
spark.sql(query).show()
print(spark.sql(query).count())

+--------------------+--------------------+
|             authors|          categories|
+--------------------+--------------------+
|Ileana Streinu an...|       math.CO cs.CG|
|        David Callan|             math.CO|
|  Sergei Ovchinnikov|             math.CO|
|Clifton Cunningha...|     math.NT math.AG|
|        Koichi Fujii|     math.CA math.AT|
|         Norio Konno|     math.PR math.AG|
|Simon J.A. Malham...|             math.NA|
|Robert P. C. de M...|             math.RA|
|  P\'eter E. Frenkel|     math.CA math.PR|
|          Mihai Popa|     math.OA math.FA|
|   Debashish Goswami|math.QA math-ph m...|
|      Mikkel {\O}bro|             math.CO|
|Nabil L. Youssef,...|       math.DG gr-qc|
|         Boris Rubin|             math.FA|
|         A. I. Molev|     math.AG math.CO|
| Branko J. Malesevic|     math.NT math.CV|
|   John W. Robertson|             math.DS|
|     Yu.N. Kosovtsov|     math-ph math.MP|
|        Osamu Fujino|             math.AG|
|Stephen C. Power ...|          

304590


In [293]:
# Get licences with 5 or more letters in the "abstract"
regexp1 = r"([a-z A-Z]{5,})"

def fiveOrMoreLetters(x):
    if str(x["abstract"]):
        r = re.search(regexp1,x["abstract"])
        if r:
            return x["license"]
        else:
            return None
    else:
        return None

in_js = { 'license': 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
          'abstract': '  Given an abelian algebraic group $A$ over a global field $F$, $\\alpha \\in\nA(F)$.\n'}
fiveOrMoreLetters(in_js)

'http://arxiv.org/licenses/nonexclusive-distrib/1.0/'

In [294]:
df_license = df.withColumn("regxp", lit(regexp1)).select(df.license).where(regexp("abstract","regxp"))

In [295]:
df_license.show()

+--------------------+
|             license|
+--------------------+
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
+--------------------+
only showing top 20 rows



In [296]:
# 2nd method
query = """select license from Archive
            where abstract REGEXP "([a-z A-Z]{5,})"
"""
spark.sql(query).show()

+--------------------+
|             license|
+--------------------+
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|http://arxiv.org/...|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
|             Unknown|
+--------------------+
only showing top 20 rows



In [297]:
# extract the statistics of numbers of pages
spark.udf.register("extract_num_pages", extract_num_pages, IntegerType())
query = """select extract_num_pages(comments) as num_pages from Archive
            where abstract is not null
"""
df_num_pages = spark.sql(query)
df_num_pages.show()
df_num_pages.createOrReplaceTempView("NumPages")
query1 = """select sum(num_pages)/count(*) as avg from NumPages
            where num_pages != 0
"""
spark.sql(query1).show()
query2 = """select AVG(extract_num_pages(comments)) as avg, SUM(extract_num_pages(comments)) as sum,
            STD(extract_num_pages(comments)) as std
            from Archive
            where license = 'Unknown'
"""
spark.sql(query2).show()

+---------+
|num_pages|
+---------+
|       37|
|        0|
|       23|
|       11|
|        6|
|       16|
|        0|
|       36|
|       14|
|       18|
|       22|
|       17|
|       10|
|       20|
|        6|
|       21|
|        5|
|       20|
|        4|
|        0|
+---------+
only showing top 20 rows



+-----------------+
|              avg|
+-----------------+
|17.85319004286046|
+-----------------+



+------------------+-------+------------------+
|               avg|    sum|               std|
+------------------+-------+------------------+
|13.368011068572079|5642584|16.777518213632323|
+------------------+-------+------------------+



In [306]:
#  'versions': [{'version': 'v1', 'created': 'Sat, 31 Mar 2007 02:26:18 GMT'},
   # {'version': 'v2', 'created': 'Sat, 13 Dec 2008 17:26:00 GMT'}],

# Question 9: Get the average number of pages per day for version 1

df.select(extract_num_pages(col("comments")).alias('avg_pages_per_day_v1'), avg('avg_pages_per_day_v1') ).where(get_json_object(col("versions")[0], "$.version") == "v1").show()


AttributeError: 'int' object has no attribute 'alias'

In [316]:
# extract day from created field
def extract_day(created_datetime):
    if created_datetime:
        return created_datetime.split(',')[0]
    else:
        return None

extract_day('Thu, 5 Apr 2007 04:48:39 GMT')

'Thu'

In [318]:
spark.udf.register("extract_day", extract_day, StringType())

<function __main__.extract_day(created_datetime)>

In [321]:

query_avg_pages_per_day = """select AVG(extract_num_pages(comments)) as avg_pages_per_day_v1 , extract_day(get_json_object(versions[0], "$.created")) as day
                             from Archive
                             where get_json_object(versions[0], "$.version") == "v1"
                             group by day
"""
spark.sql(query_avg_pages_per_day).show(truncate=False)

+--------------------+---+
|avg_pages_per_day_v1|day|
+--------------------+---+
|13.438900236335833  |Sun|
|13.647621634042908  |Mon|
|13.554587235518557  |Thu|
|13.063888425848752  |Sat|
|13.473081449845305  |Wed|
|13.470779008622024  |Tue|
|13.418742136828348  |Fri|
+--------------------+---+

